<a href="https://colab.research.google.com/github/lizjwh/groupIAI_5take2/blob/main/5%20Completely%20New%20Model/Lizzys_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Convolutional neural networks for MNIST

Below, I've coded up an utterly dreadful CNN for MNIST.  But its really, really bad (45% accuracy after 5 epochs).

You should try to improve the model!  Here are some key suggestions:
* Try a deeper network with more convolutional layers.
* Try some padding.
* Try some layers with `stride=2`.
* Try some pooling layers.
* Try changing the optimizer to Adam.
* Try tuning the learning rate.
* Try taking inspiration from the LeNet-5 architecture in the notes.

Other points:
* I'd advise keeping the final `nn.AdaptiveAvgPool2d(1)` layer.
* If you have too deep a model, with too many layers (especially layers with `stride=2`, or pooling layers), then you might find the model failing.

You might well want to check out the docs for:
* [Conv2d](https://pytorch.org/docs/stable/generated/torch.nn.Conv2d.html)
* [MaxPool2d](https://pytorch.org/docs/stable/generated/torch.nn.MaxPool2d.html#maxpool2d)
* [AvgPool2d](https://pytorch.org/docs/stable/generated/torch.nn.AvgPool2d.html#avgpool2d)

The notebook will be faster on GPU, but its still perfectly fine on CPU.  Remember, to switch to GPU, go to "Runtime" -> "Change Runtime Type".

In [1]:
!pip install torchvision
!pip install kaggle
!pip install tqdm
!pip install colorama
!pip install split-folders

In [2]:
import torch
import torchvision
import torchvision.transforms as transforms
torch.manual_seed(0)
from torch import nn, optim, tensor
from torchvision import transforms, models ,datasets
from torch.utils.data import ConcatDataset
import numpy as np
import cv2
import matplotlib.pyplot as plt
import glob
from mpl_toolkits.axes_grid1 import ImageGrid
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import auc
import os
import splitfolders

In [3]:
# Check whether we have a GPU.  Use it if we do.
device = torch.device('cpu')

In [4]:
! export KAGGLE_USERNAME="emmet454" && export KAGGLE_KEY="ee00fbc0728a71f5c5f712029e3ef004" && kaggle datasets download --force --unzip emilyburt/intro-to-ai

100% 4.08G/4.08G [03:09<00:00, 24.5MB/s]
100% 4.08G/4.08G [03:09<00:00, 23.2MB/s]


In [5]:
input_folder = 'Dataset/Full Mobility/Left hand'

#split with a ratio (train, val, test)
splitfolders.ratio(input_folder, output='Full Mobility/Left hand/',
                  seed=42, ratio=(.8, .15, .05),
                  group_prefix=None)

input_folder = 'Dataset/Full Mobility/Right hand'

#split with a ratio
splitfolders.ratio(input_folder, output='Full Mobility/Right hand/',
                  seed=42, ratio=(.8, .15, .05),
                  group_prefix=None)

input_folder = 'Dataset/Restricted mobility/Left hand'

#split with a ratio
splitfolders.ratio(input_folder, output='Restricted Mobility/Left hand/',
                  seed=42, ratio=(.8, .15, .05),
                  group_prefix=None)

input_folder = 'Dataset/Restricted mobility/Right hand'

#split with a ratio
splitfolders.ratio(input_folder, output='Restricted Mobility/Right hand/',
                  seed=42, ratio=(.8, .15, .05),
                  group_prefix=None)

Copying files: 492 files [00:04, 108.22 files/s]
Copying files: 502 files [00:04, 111.60 files/s]
Copying files: 484 files [00:04, 100.19 files/s]
Copying files: 495 files [00:04, 100.29 files/s]


In [6]:
def create_dataset(root_dir):
    """
    Creates an ImageFolder dataset from the specified root directory.
    """
    data_transforms = transforms.Compose([transforms.Resize((224, 224)),
                                          transforms.ToTensor(),
                                          transforms.RandomRotation(30),
                                          transforms.RandomHorizontalFlip(p=0.3),
                                          transforms.Normalize([0.485, 0.456, 0.406],
                                                               [0.229, 0.224, 0.225])])
    dataset = datasets.ImageFolder(root_dir, transform=data_transforms)
    return dataset

def combine_datasets(root_folders):
    """
    Combines datasets from multiple root folders into one dataset.
    """
    datasets = [create_dataset(folder) for folder in root_folders]
    combined_dataset = ConcatDataset(datasets)

    # Extract class-to-index mappings from individual datasets and merge them
    class_to_idx = {}
    for dataset in datasets:
        class_to_idx.update(dataset.class_to_idx)

    return combined_dataset, class_to_idx


def main():
  #choose which folders to include in test here
    root_folders = ['/content/Restricted Mobility/Left hand/train',
                    '/content/Restricted Mobility/Right hand/train',
                    '/content/Full Mobility/Left hand/train',
                    '/content/Full Mobility/Right hand/train']

    combined_dataset, combined_dataset.class_to_idx = combine_datasets(root_folders)

    print(f"Class to index mapping: {combined_dataset.class_to_idx}")
    return combined_dataset

def main_val():
  #choose which folders to include in test here
    root_folders = ['/content/Restricted Mobility/Left hand/val',
                    '/content/Restricted Mobility/Right hand/val',
                    '/content/Full Mobility/Left hand/val',
                    '/content/Full Mobility/Right hand/val']

    combined_dataset, combined_dataset.class_to_idx = combine_datasets(root_folders)

    print(f"Class to index mapping: {combined_dataset.class_to_idx}")
    return combined_dataset

def main_test():
  #choose which folders to include in test here
    root_folders = ['/content/Restricted Mobility/Left hand/test',
                    '/content/Restricted Mobility/Right hand/test',
                    '/content/Full Mobility/Left hand/test',
                    '/content/Full Mobility/Right hand/test']

    combined_dataset, combined_dataset.class_to_idx = combine_datasets(root_folders)

    print(f"Class to index mapping: {combined_dataset.class_to_idx}")
    return combined_dataset

combined_train_dataset = main()
combined_val_dataset = main_val()
combined_test_dataset = main_test()

Class to index mapping: {'A': 0, 'B': 1, 'C': 2, 'D': 3, 'E': 4, 'F': 5, 'G': 6, 'H': 7, 'I': 8, 'J': 9, 'K': 10, 'L': 11, 'M': 12, 'N': 13, 'O': 14, 'P': 15, 'Q': 16, 'R': 17, 'S': 18, 'Space': 19, 'T': 20, 'U': 21, 'V': 22, 'W': 23, 'X': 24, 'Y': 25, 'Z': 26}
Class to index mapping: {'A': 0, 'B': 1, 'C': 2, 'D': 3, 'E': 4, 'F': 5, 'G': 6, 'H': 7, 'I': 8, 'J': 9, 'K': 10, 'L': 11, 'M': 12, 'N': 13, 'O': 14, 'P': 15, 'Q': 16, 'R': 17, 'S': 18, 'Space': 19, 'T': 20, 'U': 21, 'V': 22, 'W': 23, 'X': 24, 'Y': 25, 'Z': 26}
Class to index mapping: {'A': 0, 'B': 1, 'C': 2, 'D': 3, 'E': 4, 'F': 5, 'G': 6, 'H': 7, 'I': 8, 'J': 9, 'K': 10, 'L': 11, 'M': 12, 'N': 13, 'O': 14, 'P': 15, 'Q': 16, 'R': 17, 'S': 18, 'Space': 19, 'T': 20, 'U': 21, 'V': 22, 'W': 23, 'X': 24, 'Y': 25, 'Z': 26}


In [7]:
# MNIST train and test datasets.  I'm not going to talk about these in this course.
# However, note that I'm using a much bigger batch size at test-time.  That's
# because at training time, we have to backprop, so we have to save all the
# intermediate variables, which takes alot of memory.  We don't have to do that
# at test-time.
train_loader = torch.utils.data.DataLoader(dataset=combined_train_dataset,
                                           batch_size=50,
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=combined_test_dataset,
                                          batch_size=50,
                                          shuffle=False)

############################################
#### Tweak the network architecture !!! ####
############################################
input_size = 784
hidden_size = 500
num_classes = 27

model = nn.Sequential(
    nn.Conv2d(in_channels=3, out_channels=100, kernel_size=3, padding=1),  #Output shape is [N, 100, H, W]
    nn.ReLU(),                                                             #Output shape is [N, 100, H, W]
    nn.Conv2d(in_channels=100, out_channels=27, kernel_size=3, padding=1), #Output shape is [N,  10, H, W]
    nn.AdaptiveAvgPool2d(1)                                                #output shape is [N,  10, 1, 1]; does global average pooling
).to(device)

#################################
#### Tweak the optimizer !!! ####
#################################
opt = torch.optim.SGD(model.parameters(), lr=0.001)

def train():
    # Does one training epoch (i.e. one pass over the data.)
    for images, labels in train_loader:
        # Move tensors to the configured device, and convert image to vector.
        images = images.to(device)
        labels = labels.to(device)

        # Forward pass
        logits = model(images).squeeze((-1, -2)) #output shape is [N, 10]

        # Backpropagation and optimization
        loss = nn.functional.cross_entropy(logits, labels)
        loss.backward()
        opt.step()
        opt.zero_grad()

def test(epoch):
    # Do one pass over the test data.
    # In the test phase, don't need to compute gradients (for memory efficiency)
    with torch.no_grad():
        correct = 0
        total = 0
        for images, labels in test_loader:
            #Convert image pixels to vector
            images = images.to(device)
            labels = labels.to(device)

            # Forward pass
            logits = model(images).squeeze((-1, -2))

            # Compute total correct so far
            predicted = torch.argmax(logits, -1)
            correct += (predicted == labels).sum().item()
            total += labels.size(0)
        print(f'Test accuracy after {epoch+1} epochs: {100 * correct / total} %')


# Run training
for epoch in range(3):
    train()
    test(epoch)

Test accuracy after 1 epochs: 3.8095238095238093 %
Test accuracy after 2 epochs: 3.8095238095238093 %


KeyboardInterrupt: 